In [1]:
from datasets import load_dataset, Dataset
import datasets
from random import randrange
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM,TrainingArguments,pipeline, BitsAndBytesConfig
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model, AutoPeftModelForCausalLM, PeftModel
from trl import SFTTrainer

import os
from dotenv import load_dotenv

load_dotenv()

HF_READ_TOKEN = os.getenv("HF_READ_TOKEN")
HF_WRITE_TOKEN = os.getenv("HF_WRITE_TOKEN")

c:\Users\kimbo\code\insight_text_game\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
current_dir = os.getcwd()
gemma_model_dir = os.path.join(current_dir, "../gemma-2-2b")
gemma_model = "google/gemma-2-2b"

In [3]:
from huggingface_hub import login
login(token=HF_READ_TOKEN)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to C:\Users\kimbo\.cache\huggingface\token
Login successful


In [4]:
base_model = AutoModelForCausalLM.from_pretrained(
    gemma_model,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16
)
new_model = "../gemma-2-2b-talkingDown-ko"
model = PeftModel.from_pretrained(base_model, new_model) # LoRA 가중치를 가져와 기본 모델에 통합

Loading checkpoint shards: 100%|██████████| 3/3 [00:12<00:00,  4.15s/it]
The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.


In [5]:
model = model.merge_and_unload()

In [6]:
# 사전 훈련된 토크나이저를 다시 로드
tokenizer = AutoTokenizer.from_pretrained(gemma_model, trust_remote_code=True)

# 토크나이저의 패딩 토큰을 종료 토큰(end-of-sentence token)과 동일하게 설정
tokenizer.pad_token = tokenizer.eos_token

# 패딩을 시퀀스의 오른쪽에 적용
tokenizer.padding_side = "right"

eos_token_id = tokenizer.eos_token_id

In [7]:
query = "저는 카페테리아에서 제 평범한 샌드위치 한 입 꺼내요. 그리고 요이치와 평범한 작은 대화를 나눴습니다."
format = f"<start_of_turn>user\n{query}<end_of_turn>\n<start_of_turn>model\n"
text_gen = pipeline(
    task="text-generation",
    model=model,
    tokenizer=tokenizer)
output = text_gen(format, max_new_tokens=50, eos_token_id=eos_token_id)
print(output[0]['generated_text'])

<start_of_turn>user
저는 카페테리아에서 제 평범한 샌드위치 한 입 꺼내요. 그리고 요이치와 평범한 작은 대화를 나눴습니다.<end_of_turn>
<start_of_turn>model
저는 카페테리아에서 제 평범한 샌드위치 한 입 꺼내요. 그리고 요이치와 평범한 작은 대화를 나눴습니다.
 podjela
저는 카페테


In [11]:
query = "저는 카페테리아에서 제 평범한 샌드위치 한 입 꺼내요. 그리고 요이치와 평범한 작은 대화를 나눴습니다."
format = f"<start_of_turn>user\n{query}<end_of_turn>\n<start_of_turn>model\n"

input_ids = tokenizer.encode(format, return_tensors='pt')

text_gen = pipeline(
    task="text-generation",
    model=model,
    tokenizer=tokenizer)

output_ids = text_gen(input_ids, max_new_tokens=50, eos_token_id=eos_token_id)
output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
print(output_text)

KeyboardInterrupt: 